In [1]:
import numpy as np
from numpy.lib import stride_tricks

from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib.pyplot import Polygon
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import mocalum as mc
import pandas as pd
import xarray as xr
import time

from pyconturb import gen_turb, gen_spat_grid  # generate turbulence, useful helper
from pyconturb.sig_models import iec_sig  # IEC 61400-1 turbulence std dev
from pyconturb.spectral_models import kaimal_spectrum  # Kaimal spectrum
from pyconturb.wind_profiles import constant_profile, power_profile  # wind-speed profile functions  

In [2]:
def sliding_window_slicing(a, no_items, item_type=0):
    """This method perfoms sliding window slicing of numpy arrays

    Parameters
    ----------
    a : numpy
        An array to be slided in subarrays
    no_items : int
        Number of sliced arrays or elements in sliced arrays
    item_type: int
        Indicates if no_items is number of sliced arrays (item_type=0) or
        number of elements in sliced array (item_type=1), by default 0

    Return
    ------
    numpy
        Sliced numpy array
    """
    if item_type == 0:
        no_slices = no_items
        no_elements = len(a) + 1 - no_slices
        if no_elements <=0:
            raise ValueError('Sliding slicing not possible, no_items is larger than ' + str(len(a)))
    else:
        no_elements = no_items                
        no_slices = len(a) - no_elements + 1
        if no_slices <=0:
            raise ValueError('Sliding slicing not possible, no_items is larger than ' + str(len(a)))

    subarray_shape = a.shape[1:]
    shape_cfg = (no_slices, no_elements) + subarray_shape
    strides_cfg = (a.strides[0],) + a.strides
    as_strided = np.lib.stride_tricks.as_strided #shorthand
    return as_strided(a, shape=shape_cfg, strides=strides_cfg)


def _get_max_turbbox_width(ffield_bbox_cfg):
    x_len = ffield_bbox_cfg['x']['max']-ffield_bbox_cfg['x']['min']
    y_len = ffield_bbox_cfg['y']['max']-ffield_bbox_cfg['y']['min']    
    return ((x_len)**2+(y_len)**2)**(.5)
    

def _get_bbox_pts(ffield_bbox_cfg):
    bbox_pts = np.full((4,2), np.nan)
    bbox_pts[0] = np.array([ffield_bbox_cfg['x']['min'], ffield_bbox_cfg['y']['min']])
    bbox_pts[1] = np.array([ffield_bbox_cfg['x']['min'], ffield_bbox_cfg['y']['max']])    
    bbox_pts[2] = np.array([ffield_bbox_cfg['x']['max'], ffield_bbox_cfg['y']['max']])        
    bbox_pts[3] = np.array([ffield_bbox_cfg['x']['max'], ffield_bbox_cfg['y']['min']])            
    
    return bbox_pts


def _rot_matrix(wind_dir):

    azimuth = wind_dir - 90  # it is revers since something is fishy with dot product
    azimuth = np.radians(azimuth) # converts to radians    
    c, s = np.cos(azimuth), np.sin(azimuth)
    R = np.array([[c, s], [-s, c]])
    return R


def _get_turbbbox(ffield_bbox_cfg, wind_dir):
    bbox_pts = _get_bbox_pts(tmp.data.ffield_bbox_cfg)
    
    # Normalize points to be at the coordinate system center
    bbox_pts_norm =bbox_pts - bbox_pts.mean(axis=0)
    
    # Find position of normalized points in future turb box coord sys
    pts_new_cord = bbox_pts_norm.dot(_rot_matrix(wind_dir)) 
    
    # Find minimum turb bounding box
    x_min, y_min = pts_new_cord.min(axis = 0)
    x_max, y_max = pts_new_cord.max(axis = 0)

    tbox_pts = np.full((4,2), np.nan)
    tbox_pts[0] = np.array([x_min, y_min])
    tbox_pts[1] = np.array([x_min, y_max])    
    tbox_pts[2] = np.array([x_max, y_max])        
    tbox_pts[3] = np.array([x_max, y_min])
    
    return tbox_pts


def _get_prob_cords(probing):
    
    x = probing.x.values[:probing.no_los.values]
    y = probing.y.values[:probing.no_los.values]    

    return np.array([x,y]).transpose()


def _calc_res(a):
    
    a.sort()
    steps = np.abs(np.roll(a,1) - a)[1:]

    return int(steps.mean())


In [3]:
koshava_xyz = [0, 0, 0]
no_scans = 10000
# wind field setup
ref_height = 100      # power law reference height
meas_height = 100 
shear_exponent = 0.2  # power law shear exponent
wind_speed = 15       # wind speed at referenec height
wind_dir = 340        # wind direction
w = 0

# beam steering setup
distance = 1000   # meter
elevation = np.degrees(np.arcsin(meas_height / distance))
angular_res = 1   # degree
azimuth_mid = 0   # central azimuth angle
sector_size = 90  # degree
scan_speed = 1    # degree.s^-1
max_speed = 50    # degree.s^-1
max_acc = 100     # degree.s^-2

# Uncertainty terms
no_sim = 10000 # number of simulations
corr_coef = 0   # correlation coefficient
mu = 0         # we assume no systematic uncertainty
azim_std = 0.1 # degrees
elev_std = 0.1 # degrees
dis_std = 10   # meters
rad_std = 0.1  # m.s-1  In [23]:  

tmp = mc.Mocalum()
# tmp.x_res = tmp.y_res = 10
  
tmp.set_ivap_probing(koshava_xyz, sector_size, azimuth_mid, angular_res, elevation, distance, 
                    no_scans, scan_speed, max_speed,max_acc)

tmp.gen_unc_contributors(corr_coef)
# tmp.gen_plaw_ffield(wind_speed, wind_dir, w, ref_height, shear_exponent)
# tmp.calc_los_speed()
# tmp.reconstruct_wind()


In [4]:
tmp.x_res = tmp.y_res = 25

tmp.z_res = 5

tmp.z_res

5

In [5]:
tmp.gen_turb_ffield(wind_speed, wind_dir, w, ref_height, shear_exponent)

# tmp.data.turb_bbox_cfg

In [9]:
tmp.data.turb_bbox_cfg

{'x': {'min': -420.1749487419361, 'max': 420.1749487419361, 'res': 50},
 'y': {'min': -748.0961557805111, 'max': 748.0961557805111, 'res': 50},
 'z': {'min': 91.19129799045851, 'max': 109.60496166144469, 'res': 5},
 't': {'min': 0.0, 'max': 596.6666666666667, 'res': 3.3333333333333335}}

In [13]:
tmp.data.tfield

<xarray.Dataset>
Dimensions:  (t: 180, y: 33, z: 7)
Coordinates:
  * t        (t) float64 0.0 3.333 6.667 10.0 13.33 ... 586.7 590.0 593.3 596.7
  * y        (y) float64 -798.1 -748.1 -698.1 -648.1 ... 651.9 701.9 751.9 801.9
  * z        (z) float64 86.19 91.19 96.19 101.2 106.2 111.2 116.2
Data variables:
    u        (z, y, t) float64 13.32 13.61 16.37 16.04 ... 14.97 12.44 14.06
    v        (z, y, t) float64 0.5014 2.155 2.311 ... -0.7236 2.564 0.02925
    w        (z, y, t) float64 1.928 0.5873 -2.84 ... -0.7368 -1.138 -1.734
Attributes:
    title:      Turbulent flow field dataset
    generator:  PyConTurb

In [ ]:
# polygon = Polygon(bbox_pts_norm,alpha=0.4, color='black')
diag = _get_max_turbbox_width(tmp.data.ffield_bbox_cfg)
bbox_pts = _get_bbox_pts(tmp.data.ffield_bbox_cfg)
bbox_pts_norm = bbox_pts - bbox_pts.mean(axis=0)

arc_cords= _get_prob_cords(tmp.data.probing) - bbox_pts.mean(axis=0)

tbox_pts = _get_turbbbox(tmp.data.ffield_bbox_cfg, wind_dir)

# coordinates in turbox coord sys
pts_new_cord = bbox_pts_norm.dot(_rot_matrix(wind_dir))

wind_dir_pt = mc.spher2cart(wind_dir,0,diag/2)[:2]
bbox = Polygon(bbox_pts_norm,alpha=0.4, color='red')
tbox = Polygon(tbox_pts.dot(_rot_matrix(-wind_dir)),alpha=0.4, color='green')
# tbox2 = Polygon(tbox_pts.dot(_rot_bbox(wind_dir)),alpha=0.4, color='yellow')

# polygon = Polygon(bbox_pts.dot(R))

fig, ax = plt.subplots(figsize=(7.5, 7.5))
plt.grid()
ax.add_patch(bbox)
ax.add_patch(tbox)

plt.scatter(bbox_pts_norm[:,0],bbox_pts_norm[:,1], c="black")
plt.scatter(arc_cords[:,0], arc_cords[:,1], c="blue")
ax.set_aspect('equal')
ax.annotate("", xy=(wind_dir_pt[0], wind_dir_pt[1]), xytext=(0, 0),arrowprops=dict(arrowstyle="<-",color='red'))

ax.set_xlim(-diag,diag)
ax.set_ylim(-diag,diag)
plt.show()

In [ ]:
arc_cords= _get_prob_cords(tmp.data.probing)
arc_cords_tbox_cs = arc_cords.dot(_rot_matrix(wind_dir))
# arc_cords_tbox_cs[:,0].sort()
# arc_cords_tbox_cs[:,1].sort()

In [ ]:
ffield_bbox_cfg = tmp.data.ffield_bbox_cfg

In [ ]:
# prepare 3D mesh for turb generation

min_step_xy = 50 # resolution in x and y

z = np.arange(ffield_bbox_cfg['z']['min'] - ffield_bbox_cfg['z']['res']*5,
               ffield_bbox_cfg['z']['max'] + 2*ffield_bbox_cfg['z']['res']*5,
               ffield_bbox_cfg['z']['res']*5)


y_step_res = _calc_res(arc_cords_tbox_cs[:,1])
y_step = min_step_xy if y_step_res <= min_step_xy else y_step_res
y = np.arange(tbox_pts[:,1].min() - y_step,
              tbox_pts[:,1].max() + 2*y_step, y_step)


x_step_res = _calc_res(arc_cords_tbox_cs[:,0])
x_step = min_step_xy if x_step_res <= min_step_xy else x_step_res
x = np.arange(tbox_pts[:,0].min() - x_step,
              tbox_pts[:,0].max() + 2*x_step, x_step)


t_step = x_step / wind_speed
T=600
t_coord = np.arange(0, T, t_step)


spat_df = gen_spat_grid(y, z)
spat_np = spat_df.to_numpy()
y_coord = spat_np[2][0::3]
z_coord = spat_np[3][0::3]


# x = np.arange(tbox_pts[:,0].min() - 25,
#               tbox_pts[:,0].max() + 25*2, 
#               [25 if _calc_res(arc_cords_tbox_cs[:,1]) <= 25 else _calc_res(arc_cords_tbox_cs[:,1])])

# x

In [ ]:
from pyconturb.wind_profiles import power_profile

In [ ]:
# %%time

turb_df = gen_turb(spat_df, T=T, dt=t_step, wsp_func = power_profile, 
                   u_ref=wind_speed, z_ref=ref_height, alpha=0.1)

In [ ]:
%%time
turb_np = turb_df.to_numpy().transpose().ravel()
turb_np = turb_np.reshape(int(len(turb_np)/len(t_coord)), len(t_coord))

u_comp = turb_np[0::3].reshape(len(y), len(z) ,len(t_coord))
v_comp = turb_np[1::3].reshape(len(y), len(z) ,len(t_coord))
w_comp = turb_np[2::3].reshape(len(y), len(z) ,len(t_coord))

In [ ]:
%%time
tubffield = xr.Dataset({'u': (['z','y', 't'], u_comp.transpose(1, 0, 2)),
                        'v': (['z','y','t'], v_comp.transpose(1, 0, 2)),
                        'w': (['z', 'y', 't'], w_comp.transpose(1, 0, 2))},
                        coords={'y': y,
                                'z': z,
                                't': t_coord})

In [ ]:
tubffield.u.mean(axis=2).plot()

In [ ]:
tubffield.u.mean(axis=2).plot()

In [ ]:
tmp.data.ffield_bbox_cfg

In [ ]:
"""Utility functions for notebook examples"""
import matplotlib.pyplot as plt


def plot_slice(spat_df, turb_df, comp='u', val='mean',
               interpolation='none', ax=None):
    """Plot a time or statistic slice of turbulence box (grid only).
    """
    y = spat_df.loc['y'].unique()
    z = spat_df.loc['z'].unique()
    if comp not in ['u', 'v', 'w']:
        raise ValueError('Component must be "u", "v" or "w"!')
    if isinstance(val, str):
        plot_grid = (turb_df.filter(regex=comp).describe().loc[val]
                     .values.reshape(y.size, z.size).T)
    elif isinstance(val, (int, float)):
        if val not in turb_df.index:
            raise ValueError('Requested slice not in index!')
        plot_grid = turb_df.filter(regex=comp).loc[val].values.reshape(y.size, z.size)
    if ax is None:
        fig, ax = plt.subplots(figsize=(7.5, 7.5))
    plt.imshow(plot_grid,  # imshow requires nz-ny slice
               origin='lower',  # smallest y-z in lower left, not upper left
#                extent=[y[0], y[-1], z[0], z[-1]],  # lateral and vertical limits
               interpolation=interpolation)  # image smoothing
    plt.colorbar()
    return ax


def plot_interp(yp, zp, valp, y, z, val, fmt='%.2f'):
    """a useful plotting function for the interpolator examples"""
    plt.imshow(val.reshape(y.shape), origin='lower',
               extent=[y.min(), y.max(), z.min(), z.max()], cmap='Greys')
    plt.colorbar(format=fmt)
    plt.scatter(yp, zp, s=100, c=valp, edgecolors='0.8', cmap='Greens', label='intp. locs')
    plt.colorbar(format=fmt)

In [ ]:
y = spat_df.loc['y'].unique()
z = spat_df.loc['z'].unique()
plot_grid = turb_df.filter(regex='u').loc[0].values.reshape(y.size, z.size).T


In [ ]:
fig, ax = plt.subplots(figsize=(7.5, 7.5))

plt.imshow(plot_grid,  # imshow requires nz-ny slice
               origin='lower',  # smallest y-z in lower left, not upper left
    
               interpolation='none')  # image smoothing


In [ ]:
t = 0
x = plot_slice(spat_df, turb_df, val=0)
ax.set_title(f'Turbulence slice at t = {t}');

In [ ]:
# pulls bbox corners

x_bb_min=tmp.data.ffield_bbox_cfg['x']['min']
x_bb_max=tmp.data.ffield_bbox_cfg['x']['max']
y_bb_min=tmp.data.ffield_bbox_cfg['y']['min']
y_bb_max=tmp.data.ffield_bbox_cfg['y']['max']
z_bb_min=tmp.data.ffield_bbox_cfg['z']['min']
z_bb_max=tmp.data.ffield_bbox_cfg['z']['max']
x_bb_c=x_bb_min+(x_bb_max-x_bb_min)/2
y_bb_c=y_bb_min+(y_bb_max-y_bb_min)/2
w_tbox=((x_bb_max-x_bb_min)**2+(y_bb_max-y_bb_min)**2)**(.5)

In [ ]:
w_tbox

In [ ]:
######### Turbulence box code starts here ###################################
### input ##### x = longitudian y =lateral z= same
x_dens=300 #max 60 on my dell
y_dens=5
z_dens=3
dt_in=2 #max 10 on my dell
T_in=x_dens*dt_in
u_ref_in=10
wdir=330

###############
y = np.linspace(-w_tbox/2, w_tbox/2, y_dens)  # 11 lateral points from -50 to 50 (center @ 0)
z = np.linspace(z_bb_min, z_bb_max, z_dens) # 13 vertical points from 40 to 160 (center @ 100)

spat_df = gen_spat_grid(y, z)

turb_df = gen_turb(spat_df, T=2*T_in, dt=dt_in, u_ref=u_ref_in)

x=np.linspace(-T_in*u_ref_in,0, x_dens) #dt_in number of longitudinal points from 0 m to T_in*u_ref_in meters
# xs, ys, zs, = np.meshgrid(x, y, z)
# xs3=np.repeat(xs, 3, axis=2)
# results=np.array(xs3).flatten() 

# creating 3D gen spat usying modified jenny function
comps=[0, 1, 2]
_spat_rownames = ['k', 'x', 'y', 'z']
xs,ys, zs = np.meshgrid(x, y, z)  # make a 3d meshgrid
ks = np.array(comps, dtype=int)  # sanitizing
col_names = [f'{"uvw"[k]}_p{ip}' for ip in range(xs.size) for k in ks]
# spat_arr = np.c_[np.tile(comps, xs.size),
#                   np.repeat(np.c_[xs.ravel(), ys.T.ravel(), zs.T.ravel()],
#                             ks.size, axis=0)].T  # create array using numpy
spat_arr = np.c_[np.tile(comps, xs.size),
                  np.repeat(np.c_[xs.transpose(1,0,2).ravel(),ys.transpose(1,0,2).ravel(),zs.transpose(0,1,2).ravel()],
                            ks.size, axis=0)].T
spat_df3d=pd.DataFrame(spat_arr, index=_spat_rownames, columns=col_names)


# spat_df.loc['x']=np.array(xs3).flatten() 

# adapting turb_df to 3D (frozen turbulence assumption)
turb3d=turb_df.values # df to numpy to make things easier
temp=np.full((x_dens, x_dens*turb3d.shape[1]), np.nan)
for rown in range(x_dens):
    temp[rown]=turb3d[rown:rown+x_dens,:].ravel() 

for rown in range(temp.shape[0]): #appending 3D turbulence to 3D grid
    spat_df3d.loc[turb_df.index[rown],:] = temp[rown,:]
# del temp
#defying rotation matrix
theta = np.radians(wdir-270)
c, s = np.cos(theta), np.sin(theta)
R = np.array(((c, -s), (s, c))) # or x = pd.DataFrame(data=R,index=['X','Y'])

#trying to rotate turbulence box along wind direction
xy_df=spat_df3d.loc[['x','y']].transpose()
p =xy_df.values
rotated_array = p.dot(R)
rotated_array[:,0]=rotated_array[:,0]+x_bb_c+w_tbox/2*np.cos(theta)
rotated_array[:,1]=rotated_array[:,1]+y_bb_c-w_tbox/2*np.sin(theta)
mmmm=spat_df3d.loc['z'].transpose().values
rotated_array=np.concatenate((rotated_array,mmmm[:,None]),axis=1)

In [ ]:
xy_df

In [ ]:
spat_df3d.

In [ ]:
#interpolating - obtaining final grid
xp=tmp.data.ffield.x.values
yp=tmp.data.ffield.y.values
zp=tmp.data.ffield.z.values
XXX, YYY, ZZZ = np.meshgrid(xp,yp,zp)
XX=XXX.transpose(1,0,2).reshape(-1,XXX.shape[1])
Xp=XX.reshape(-1,1)
YY=YYY.transpose(1,0,2).reshape(-1,YYY.shape[1])
Yp=YY.reshape(-1,1)
ZZ=ZZZ.transpose(0,1,2).reshape(-1,ZZZ.shape[1])
Zp=ZZ.reshape(-1,1)
request=np.concatenate((Xp,Yp,Zp),axis=1)

In [ ]:
#rearranging dataframe for xarray conversion
spat_df3d.loc['x']=rotated_array[:,0]
spat_df3d.loc['y']=rotated_array[:,1]
spat_turb=spat_df3d.drop('k')
del spat_df3d

In [ ]:
# for u
mask_u_df = spat_turb.columns.str.contains('u_p*')
u_df=spat_turb.loc[:,mask_u_df].T
#interpolation here
points=u_df[['x','y','z']].values
values=u_df.iloc[:,3:].values
u_p=griddata(points, values, request)
u_df1=pd.DataFrame(np.concatenate((request,u_p),axis=1), columns=u_df.columns)
# for u continues
u_df_stacked = u_df1.set_index(['x', 'y', 'z']).stack().reset_index(name='u')
u_df_stacked.columns=u_df_stacked.columns.str.replace('level_3','t')
u_df_multiindex = u_df_stacked.set_index(['x', 'y', 'z', 't'])
del u_df,u_df_stacked
u_ds=u_df_multiindex.to_xarray() #main problem for memory


In [ ]:
# for v
mask_v_df = spat_turb.columns.str.contains('v_p*')
v_df=spat_turb.loc[:,mask_v_df].T
#interpolation here
points=v_df[['x','y','z']].values
values=v_df.iloc[:,3:].values
v_p=griddata(points, values, request)
v_df1=pd.DataFrame(np.concatenate((request,v_p),axis=1), columns=v_df.columns)
# for v continues
v_df_stacked = v_df1.set_index(['x', 'y', 'z']).stack().reset_index(name='v')
v_df_stacked.columns=v_df_stacked.columns.str.replace('level_3','t')
v_df_multiindex = v_df_stacked.set_index(['x', 'y', 'z', 't'])
del v_df,v_df_stacked
v_ds=v_df_multiindex.to_xarray() #main problem for memory

In [ ]:
# for w
mask_w_df = spat_turb.columns.str.contains('w_p*')
w_df=spat_turb.loc[:,mask_w_df].T
#interpolation here?
points=w_df[['x','y','z']].values
values=w_df.iloc[:,3:].values
w_p=griddata(points, values, request)
w_df1=pd.DataFrame(np.concatenate((request,w_p),axis=1), columns=w_df.columns)
# for w continues
w_df_stacked = w_df1.set_index(['x', 'y', 'z']).stack().reset_index(name='w')
w_df_stacked.columns=w_df_stacked.columns.str.replace('level_3','t')
w_df_multiindex = w_df_stacked.set_index(['x', 'y', 'z', 't'])
del w_df, w_df_stacked
w_ds=w_df_multiindex.to_xarray()

In [ ]:
# mergining three separate dataarrays
turb=xr.merge([u_ds, v_ds, w_ds])
turb.attrs['title'] = 'Flow field dataset'
turb.attrs['generator'] = 'pyconturb(Rinker et al.)'

In [ ]:
#plotting
plt.plot(spat_turb.loc['x'], spat_turb.loc['y'], 'o', color='black');plt.grid();
plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

plt.scatter(x_bb_min,y_bb_min)
plt.scatter(x_bb_min,y_bb_max)
plt.scatter(x_bb_max,y_bb_min)
plt.scatter(x_bb_max,y_bb_max)
plt.scatter(x_bb_c,y_bb_c)